In [23]:
import os

import numpy as np
import pandas as pd

In [24]:
train = open('archive (2)/train.txt', 'r')
test = open('archive (2)/test.txt', 'r')
val = open('archive (2)/val.txt', 'r')

In [25]:
train_ = []
test_ = []
val_ = []
for i in train:
    data = i.split(';')
    train_.append([data[0], data[1]])
for i in test:
    data = i.split(';')
    test_.append([data[0], data[1]])
for i in val:
    data = i.split(';')
    val_.append([data[0], data[1]])


In [26]:
train_ = pd.DataFrame(train_)
test_ = pd.DataFrame(test_)
val_ = pd.DataFrame(val_)

In [27]:
full_df = pd.concat([train_, test_, val_], axis=0)
labels = full_df[1].unique()

In [56]:
full_df

,0,1
0,"[i, didnt, feel, humiliated]",0
1,"[i, can, go, from, feeling, so, hopeless, to, ...",0
2,"[im, grabbing, a, minute, to, post, i, feel, g...",1
3,"[i, am, ever, feeling, nostalgic, about, the, ...",2
4,"[i, am, feeling, grouchy]",1
...,...,...
1995,"[im, having, ssa, examination, tomorrow, in, t...",0
1996,"[i, constantly, worry, about, their, fight, ag...",5
1997,"[i, feel, its, important, to, share, this, inf...",5
1998,"[i, truly, feel, that, if, you, are, passionat...",5


In [6]:
labels_int = {labels[i]: i for i in range(len(labels))}

In [35]:
train_[1] = train_[1].map(labels_int)
test_[1] = test_[1].map(labels_int)
val_[1] = val_[1].map(labels_int)

In [54]:
train_[0] = train_[0].map(word_tokenize)
test_[0] = test_[0].map(word_tokenize)
val_[0] = val_[0].map(word_tokenize)

In [37]:
from nltk import WordNetLemmatizer
import string
from nltk.corpus import stopwords
import nltk
import re

stops = stopwords.words('english')
punct = set(string.punctuation)
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    cleaned_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stops]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

In [38]:
from nltk import word_tokenize

full_df[0] = full_df[0].map(word_tokenize)

In [39]:
full_df[1] = full_df[1].map(labels_int)

In [40]:
full_df

,0,1
0,"[i, didnt, feel, humiliated]",0
1,"[i, can, go, from, feeling, so, hopeless, to, ...",0
2,"[im, grabbing, a, minute, to, post, i, feel, g...",1
3,"[i, am, ever, feeling, nostalgic, about, the, ...",2
4,"[i, am, feeling, grouchy]",1
...,...,...
1995,"[im, having, ssa, examination, tomorrow, in, t...",0
1996,"[i, constantly, worry, about, their, fight, ag...",5
1997,"[i, feel, its, important, to, share, this, inf...",5
1998,"[i, truly, feel, that, if, you, are, passionat...",5


In [41]:
from collections import Counter

words_counter = Counter()
for i in full_df[0]:
    for j in i:
        words_counter[j] += 1




In [84]:
words_most_common = words_counter.most_common()

In [85]:
words_most_common_indices = {words_most_common[i - 1][0]: i for i in range(1, len(words_most_common) + 1)}

In [88]:
train2int = []
for i, label in zip(train_[0], train_[1]):
    _ = []
    for j in i:
        _.append(words_most_common_indices[j])
    train2int.append([_, label])

test2int = []
for i, label in zip(test_[0], test_[1]):
    _ = []
    for j in i:
        _.append(words_most_common_indices[j])
    test2int.append([_, label])

val2int = []
for i, label in zip(val_[0], val_[1]):
    _ = []
    for j in i:
        _.append(words_most_common_indices[j])
    val2int.append([_, label])




In [78]:
# from tensorflow.keras.models import Sequential
# from keras.src.layers import Embedding, SimpleRNN, Dense
#
# model_new = Sequential()
#
# model_new.add(Embedding(17094, 8))
# model_new.add(SimpleRNN(8))
# model_new.add(Dense(1, activation='sigmoid'))
#
# model_new.compile(
#     # loss=keras.losses.BinaryCrossentropy(),
#     loss='binary_crossentropy',
#     optimizer='adam',
#     metrics=["accuracy"]
# )

In [89]:
X_train_embs = [i[0] for i in train2int]
y_train = [i[1] for i in train2int]

In [90]:
X_test_embs = [i[0] for i in test2int]
y_test = [i[1] for i in test2int]

In [146]:
from keras_preprocessing.sequence import pad_sequences

padded_x_train = pad_sequences(
    X_train_embs,
    maxlen=200,
)

padded_x_test = pad_sequences(
    X_test_embs,
    maxlen=200,
)

In [94]:
# X_test_embs_first1000 = []
# X_train_embs_first1000 = []
# for i in X_test_embs:
#     _ = [j for j in i if j <= 1000]
#     X_test_embs_first1000.append(_)
#
# for i in X_train_embs:
#     _ = [j for j in i if j <= 1000]
#     X_train_embs_first1000.append(_)


In [105]:
# import tensorflow as tf
# import numpy as np
# padded_x_train = np.array(X_train_embs_first1000)
# padded_x_test = np.array(X_test_embs_first1000)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16000,) + inhomogeneous part.

In [164]:
from keras.src.layers import Embedding, SimpleRNN

model_new = Sequential()

model_new.add(Embedding(17094, 8, input_length=200))
model_new.add(SimpleRNN(8))
model_new.add(Dense(1, activation='softmax'))

model_new.compile(
    # loss=keras.losses.BinaryCrossentropy(),
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=["accuracy"]
)

model_new.summary()
history_ = model_new.fit(padded_x_train, np.array(y_train),
                         epochs=5, verbose=1, validation_data=(padded_x_test, np.array(y_test)))

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 200, 8)            136752    
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 8)                 136       
                                                                 
 dense_36 (Dense)            (None, 1)                 9         
                                                                 
Total params: 136897 (534.75 KB)
Trainable params: 136897 (534.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
500/500 [==============================] - 12s 20ms/step - loss: -5.5034 - accuracy: 0.1349 - val_loss: -10.0111 - val_accuracy: 0.1375
Epoch 2/5
500/500 [==============================] - 10s 19ms/step - loss: -13.6072 - accuracy: 0.1349 - val_loss: -17.4086 - val_a

In [178]:
from tensorflow.keras.layers import Dense, LSTM

model_ = Sequential()
model_.add(LSTM(64, activation='relu', input_shape=(200, 1)))
model_.add(Dense(1, activation='softmax'))
model_.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])







In [179]:
# padded_x_train = tf.convert_to_tensor(padded_x_train)
# padded_x_test = tf.convert_to_tensor(padded_x_test)


In [180]:
# padded_x_train_ = np.expand_dims(padded_x_train, 2)
# padded_x_test_ = np.expand_dims(padded_x_test, 2)

In [181]:
_history = model_.fit(padded_x_train, np.array(y_train),
                      epochs=5, verbose=1, validation_data=(padded_x_test, np.array(y_test)), )

Epoch 1/5
500/500 [==============================] - 19s 37ms/step - loss: -163.2444 - accuracy: 0.1349 - val_loss: -505.3679 - val_accuracy: 0.1375
Epoch 2/5
500/500 [==============================] - 17s 34ms/step - loss: nan - accuracy: 0.1849 - val_loss: nan - val_accuracy: 0.2905
Epoch 3/5
500/500 [==============================] - 20s 40ms/step - loss: nan - accuracy: 0.2916 - val_loss: nan - val_accuracy: 0.2905
Epoch 4/5
500/500 [==============================] - 22s 43ms/step - loss: nan - accuracy: 0.2916 - val_loss: nan - val_accuracy: 0.2905
Epoch 5/5
500/500 [==============================] - 19s 38ms/step - loss: nan - accuracy: 0.2916 - val_loss: nan - val_accuracy: 0.2905


In [135]:
padded_x_test

array([[   0,    0,    0, ..., 4974,  111,   58],
       [   0,    0,    0, ...,    1,    2,  457],
       [   0,    0,    0, ...,  395,   24,   67],
       ...,
       [   0,    0,    0, ...,    7,    9, 3512],
       [   0,    0,    0, ...,   49,   10, 2678],
       [   0,    0,    0, ...,    4,  294, 1964]], dtype=int32)